# IMT‑211 · Análisis Cinemático de Mecanismos de 4 Barras

Cuaderno base para **Mecanismos**. Permite:
- Definir un mecanismo **cuadrilátero articulado** (4‑bar) en el plano.
- Simular el movimiento en el tiempo (posición, velocidad, aceleración) a partir de una entrada angular (crank).
- **Graficar cualquier variable** relevante en función del tiempo.
- **Animar** los eslabones y **trazar** la trayectoria de un punto de interés (similar a GeoGebra).

> Dependencias recomendadas del entorno (construidas en tu *repo* y entorno virtual): `python >=3.11`, `numpy`, `matplotlib`. Opcional: `ipywidgets` (para controles interactivos), `pandas` (para exportar resultados).


## Importación de librerias

In [47]:
import math
import numpy as np
import sympy as sp
from IPython.display import Math, display
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


## 1) Definición del mecanismo y del perfil de entrada

### Eslabones:
- `L0`: **Bancada**
- `L1`: **Biela**
- `L2`: **Acoplador**
- `L3`: **Brazo balancín**

### Juntas:
- `O_1`: **L0,R,L1**
- `A`:  **L1,R,L2**
- `B`:  **L2,R,L3**
- `O_4`: **L0,R,L3**

### Punto de Interés:
- `Y`: **Extremo del brazo del balancín**




### Datos de entrada:

In [9]:
# --- Longitudes de eslabones (unidades arbitrarias, p. ej., mm) ---
l0x = -32  # logitud en x de bancada
l0y = -5
l1 = 12
l2 = 30
l3 = 25

### Vectores posición

In [56]:
# --- Crear las variables angulares ---
th1 = sp.symbols('theta_1')
th2 = sp.symbols('theta_2')
th3 = sp.symbols('theta_3')

# --- definir mis verctores ---
Ro4 = sp.Matrix([[l0x],
                [l0y],
                [0]])

Ra = sp.Matrix([[l1*sp.cos(th1)],
                [l1*sp.sin(th1)],
                [0]])

Rab = sp.Matrix([[l2*sp.cos(th2)],
                 [l2*sp.sin(th2)],
                 [0]])

Ro4b = sp.Matrix([[l3*sp.cos(th3)],
                  [l3*sp.sin(th3)],
                  [0]])

Rb = Ra + Rab

print("Vectores de posición relativa:")
print("Ra:")
display(Math(sp.latex(Ra)))
print("Rab:")
display(Math(sp.latex(Rab)))
print("Ro4b:")
display(Math(sp.latex(Ro4b)))

Vectores de posición relativa:
Ra:


<IPython.core.display.Math object>

Rab:


<IPython.core.display.Math object>

Ro4b:


<IPython.core.display.Math object>

### Vectores Velocidad

In [55]:
# --- Crear las variables de velocidades angulares ---
w1 = sp.symbols('omega_1')
w2 = sp.symbols('omega_2')
w3 = sp.symbols('omega_3')

# --- Crear los Vectores de velocidad --- (para angulares y escalares)
W1 = sp.Matrix([[0],
                [0],
                [w1]])

W2 = sp.Matrix([[0],
                [0],
                [w2]])

W3 = sp.Matrix([[0],
                [0],
                [w3]])

Va = W1.cross(Ra)

Vab = W2.cross(Rab)

Vo4b = W3.cross(Ro4b)

Vb = Va + Vab
# Vb = Vo4b = W3.cross(Ro4b) ## otra forma de obtener Vb

print("Vectores de velocidad relativa:")
print("Va:")
display(Math(sp.latex(Va)))
print("Vab:")
display(Math(sp.latex(Vab)))
print("Vo4b:")
display(Math(sp.latex(Vo4b)))

Vectores de velocidad relativa:
Va:


<IPython.core.display.Math object>

Vab:


<IPython.core.display.Math object>

Vo4b:


<IPython.core.display.Math object>

### Vectores Aceleración

In [57]:
# --- Crear las variables de aceleraciones angulares ---
a1 = sp.symbols('alpha_1')
a2 = sp.symbols('alpha_2')
a3 = sp.symbols('alpha_3')

# --- Crear los Vectores de Aceleración --- (para angulares y escalares)
A1 = sp.Matrix([[0],
                [0],
                [a1]])

A2 = sp.Matrix([[0],
                [0],
                [a2]])

A3 = sp.Matrix([[0],
                [0],
                [a3]])

Aa_T = A1.cross(Ra)
Aa_N = -w1*Ra
Aa = Aa_T + Aa_N # + Aa_C ## -> la aceleración de Coriolis/ movimiento linean y rotacional

Aab_T = A2.cross(Rab)
Aab_N = -w2*Rab
Aab = Aab_T + Aab_N

Ab = Aa + Aab

Ao4b_T = A3.cross(Ro4b)
Ao4b_N = -w3*Ro4b
Ao4b = Ao4b_T + Ao4b_N

#Ab = Ao4b = Ao4b_T + Ao4b_N ## otra forma de obtener la aceleración en B

print("Vectores de aceleración relativa:")
print("Aa:")
display(Math(sp.latex(Aa)))
print("Aab:")
display(Math(sp.latex(Aab)))
print("Ao4b:")
display(Math(sp.latex(Ao4b)))

Vectores de aceleración relativa:
Aa:


<IPython.core.display.Math object>

Aab:


<IPython.core.display.Math object>

Ao4b:


<IPython.core.display.Math object>

## 2) Escribir el modelo matemático

In [58]:
# Construir ecuaciones
Q = sp.Eq(Ra + Rab, Ro4 + Ro4b)

dQ = sp.Eq(Va + Vab, Vo4b)

ddQ = sp.Eq(Aa + Aab, Ao4b)

print("\n Modelo de posición:")
display(Math(sp.latex(Q)))
print("\n Modelo de velocidad:")
display(Math(sp.latex(dQ)))
print("\n Modelo de aceleración:")
display(Math(sp.latex(ddQ)))



 Modelo de posición:


<IPython.core.display.Math object>


 Modelo de velocidad:


<IPython.core.display.Math object>


 Modelo de aceleración:


<IPython.core.display.Math object>

## 3) Resolución del modelo
Para un instante determinado, un parámetro de entrada.

### Modelo de posición

In [93]:
# dato de entrada
theta1 = sp.rad(30)

valores = {th1: theta1}

# evaluación del dato de entrada
Q_eval = Q.evalf(subs=valores)
print("Modelo evaluado:")
display(Math(sp.latex(Q_eval)))

# soluciones
sol = sp.solve(Q_eval, (th2, th3))
print("Soluciones posibles:")
display(Math(sp.latex(sol)))

# selección de soluciones
theta2 = sol[0][0]
theta3 = sol[0][1]

# Mostrar resultados
print("theta1:", theta1.evalf())
display(sp.N(theta1, 5))
print("theta2:", theta2.evalf())
display(sp.N(theta2, 5))
print("theta3:", theta3.evalf())
display(sp.N(theta3, 5))


Modelo evaluado:


<IPython.core.display.Math object>

Soluciones posibles:


<IPython.core.display.Math object>

theta1: 0.523598775598299


0.52360

theta2: 3.97888621355040


3.9789

theta3: -0.468334483520582


-0.46833

### Modelo de velocidad

In [96]:
# dato de entrada
omega1 = 300.0

valores = {th1: theta1,
           th2: theta2,
           th3: theta3,
           w1: omega1}

# evaluación del dato de entrada
dQ_eval = dQ.evalf(subs=valores)
print("Modelo evaluado:")
display(Math(sp.latex(dQ_eval)))

# soluciones
sol = sp.solve(dQ_eval, (w2, w3))
print("Solución")
display(Math(sp.latex(sol)))

# Almacenar en memoria
omega2 = sol[w2]
omega3 = sol[w3]

# Mostrar resultados
print("omega1:", omega1)
display(sp.N(omega1, 5))
print("omega2:", omega2.evalf())
display(sp.N(omega2, 5))
print("omega3:", omega3.evalf())
display(sp.N(omega3, 5))

Modelo evaluado:


<IPython.core.display.Math object>

Solución


<IPython.core.display.Math object>

omega1: 300.0


300.00

omega2: 104.088271386036


104.09

omega3: 46.0441467620822


46.044

### Modelo de aceleración

In [98]:
# dato de entrada
alpha1 = -10.0

valores = {th1: theta1,
           th2: theta2,
           th3: theta3,
           w1: omega1,
           w2: omega2,
           w3: omega3,
           a1: alpha1}

# evaluación del dato de entrada
ddQ_eval = ddQ.evalf(subs=valores)
print("Modelo evaluado:")
display(Math(sp.latex(ddQ_eval)))

# soluciones
sol = sp.solve(ddQ_eval, (a2, a3))
print("Solución")
display(Math(sp.latex(sol)))

# Almacenar en memoria
alpha2 = sol[a2]
alpha3 = sol[a3]

# Mostrar resultados
print("alpha1:", alpha1)
display(sp.N(alpha1, 5))
print("alpha2:", alpha2.evalf())
display(sp.N(alpha2, 5))
print("alpha3:", alpha3.evalf())
display(sp.N(alpha3, 5))

Modelo evaluado:


<IPython.core.display.Math object>

Solución


<IPython.core.display.Math object>

alpha1: -10.0


-10.000

alpha2: -3.46960904620127


-3.4696

alpha3: -1.53480489206926


-1.5348